In [1]:
import numpy as np
from scipy.spatial import Voronoi
import plotly.graph_objs as go

In [7]:
cell = np.array([[0.0, 0.5, 0.5],   # Primitive cell basis vectors (FCC lattice)
                 [0.5, 0.0, 0.5],
                 [0.5, 0.5, 0.0]])

icell = np.linalg.inv(cell).T
b1, b2, b3 = np.linalg.norm(icell, axis=1)   # Compute reciprocal lattice vectors

px, py, pz = np.tensordot(icell, np.mgrid[-1:2, -1:2, -1:2], axes=[0, 0])   # Generate points for Voronoi diagram
points = np.c_[px.ravel(), py.ravel(), pz.ravel()]

vor = Voronoi(points)   # Voronoi tesellation (Thiessen polygon maps)

bz_facets = []
bz_ridges = []
bz_vertices = []

for pid, rid in zip(vor.ridge_points, vor.ridge_vertices):
    if 13 in pid:   # Center of the Voronoi cell
        bz_ridges.append(vor.vertices[np.r_[rid, [rid[0]]]])
        bz_facets.append(vor.vertices[rid])
        bz_vertices += rid

bz_vertices = list(set(bz_vertices))
v = vor.vertices[bz_vertices]

x, y, z = v[:, 0], v[:, 1], v[:, 2]   # Vertices of the surface plot

fig = go.Figure(data=[go.Mesh3d(
    x=x, y=y, z=z,
    color='mediumaquamarine',
    opacity=0.6,
    alphahull=0,
    flatshading=True
)])

for edge in bz_ridges:   # Edges
    edge_x, edge_y, edge_z = edge[:, 0], edge[:, 1], edge[:, 2]
    fig.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        showlegend=False
    ))

In [8]:
high_symmetry_points = {   # Scaled units
    "Γ": [0.0000000000, 0.0000000000, 0.0000000000],
    "K": [0.3750000000, 0.3750000000, 0.7500000000],
    "L": [0.5000000000, 0.5000000000, 0.5000000000],
    "U": [0.6250000000, 0.2500000000, 0.6250000000],
    "W": [0.5000000000, 0.2500000000, 0.7500000000],
    "X": [0.5000000000, 0.0000000000, 0.5000000000]
}

# High-symmetry path
high_symmetry_path = ["K", "Γ", "L", "K", "W", "L", "U", "W", "U", "X", "W", "X", "Γ"]
path_coords = np.array([high_symmetry_points[p] for p in high_symmetry_path])

In [9]:
fig.add_trace(go.Scatter3d(
    x=path_coords[:, 0],
    y=path_coords[:, 1],
    z=path_coords[:, 2],
    mode='lines+markers',
    line=dict(color='red', width=4),
    marker=dict(size=6, color='red'),
    name='High-Symmetry Path'
))

for label, coord in high_symmetry_points.items():
    fig.add_trace(go.Scatter3d(
        x=[coord[0]],
        y=[coord[1]],
        z=[coord[2]],
        mode='markers+text',
        text=[label],
        textposition="top center",
        marker=dict(size=7, color='darkblue', symbol='circle'),
        name=f'{label} point'
    ))

In [10]:
arrow_scale = 1.5
arrowhead_size = 0.2

fig.add_trace(go.Scatter3d(
    x=[0, arrow_scale], y=[0, 0], z=[0, 0],
    mode='lines+text',
    line=dict(color='black', width=5),
    text=["", "b1"],
    textposition="top center",
    name='kx'
))
fig.add_trace(go.Scatter3d(
    x=[0, 0], y=[0, arrow_scale], z=[0, 0],
    mode='lines+text',
    line=dict(color='black', width=5),
    text=["", "b2"],
    textposition="top center",
    name='ky'
))
fig.add_trace(go.Scatter3d(
    x=[0, 0], y=[0, 0], z=[0, arrow_scale],
    mode='lines+text',
    line=dict(color='black', width=5),
    text=["", "b3"],
    textposition="top center",
    name='kz'
))

kx_end = [arrow_scale, 0, 0]
ky_end = [0, arrow_scale, 0]
kz_end = [0, 0, arrow_scale]

fig.add_trace(go.Cone(
    x=[kx_end[0]], y=[kx_end[1]], z=[kx_end[2]],
    u=[0.2], v=[0], w=[0],
    colorscale=[[0, 'black'], [1, 'black']],
    showscale=False,
    sizemode='absolute',
    sizeref=arrowhead_size
))

fig.add_trace(go.Cone(
    x=[ky_end[0]], y=[ky_end[1]], z=[ky_end[2]],
    u=[0], v=[0.2], w=[0],
    colorscale=[[0, 'black'], [1, 'black']],
    showscale=False,
    sizemode='absolute',
    sizeref=arrowhead_size
))

fig.add_trace(go.Cone(
    x=[kz_end[0]], y=[kz_end[1]], z=[kz_end[2]],
    u=[0], v=[0], w=[0.2],
    colorscale=[[0, 'black'], [1, 'black']],
    showscale=False,
    sizemode='absolute',
    sizeref=arrowhead_size
))

fig.update_layout(
    title_text='<b>First BZ of MgO</b>',
    title_x=0.5,
    titlefont=dict(size=20, color='darkslategray'),
    width=1000,
    height=900,
    scene=dict(
        xaxis=dict(
            title='x',
            titlefont=dict(size=12),
            showgrid=False,
            zeroline=False,
            showline=False,
            backgroundcolor='white',
            showticklabels=False
        ),
        yaxis=dict(
            title='y',
            titlefont=dict(size=12),
            showgrid=False,
            zeroline=False,
            showline=False,
            backgroundcolor='white',
            showticklabels=False
        ),
        zaxis=dict(
            title='z',
            titlefont=dict(size=12),
            showgrid=False,
            zeroline=False,
            showline=False,
            backgroundcolor='white',
            showticklabels=False
        ),
        aspectratio=dict(x=1, y=1, z=1),
        camera=dict(
            eye=dict(x=1.25, y=1.25, z=1.25)
        )
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    showlegend=False
)

fig.show()